In [1]:
import os
import json
import csv
import time
import asyncio
from dotenv import load_dotenv
from openai import OpenAI
from openai import AsyncOpenAI

In [25]:
system_prompt = """
You are an advanced AI assistant designed to help users on an e-commerce website focused on television sales. Your primary goal is to provide users with detailed, comprehensive, and accurate information about televisions. When responding to user inquiries, ensure your answers are in-depth, covering all aspects of their question. Include specifics like technical features, price comparisons, user reviews, and suitability for different needs (like gaming, movie watching, etc.). Tailor your responses to the user's level of understanding, and proactively address potential follow-up questions. Remember to stay updated on the latest trends and models in the television market. Your communication should be professional, yet friendly and approachable, making the information accessible to both tech-savvy and novice users. Always aim to assist users in making informed decisions about their television purchases.
"""

user_prompt = f"""
How to choose a TV?
"""

In [4]:

tools = [
    {
        "type": "function",
        "function": {
            "name": "FetchProductDetails",
            "description": "Retrieve detailed information about specific TV models, including specifications, prices, and availability.",
            "parameters": {
                "type": "object",
                "properties": {
                    "model": {
                        "type": "string",
                        "description": "The specific model or brand of the television."
                    }
                },
                "required": ["model"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "ProductFiltering",
            "description": "Suggest television models based on user preferences, budget, and specific use cases like gaming or cinema experience.",
            "parameters": {
                "type": "object",
                "properties": {
                    "preferences": {
                        "type": "string",
                        "description": "User preferences such as screen type, smart features, etc."
                    },
                    "budget": {
                        "type": "string",
                        "description": "The budget range for the television purchase."
                    },
                    "useCase": {
                        "type": "string",
                        "description": "Specific use case like gaming, cinema experience, etc."
                    }
                },
                "required": ["preferences", "budget"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "FetchUserReview",
            "description": "Summarize and present user reviews and ratings for specific television models.",
            "parameters": {
                "type": "object",
                "properties": {
                    "model": {
                        "type": "string",
                        "description": "The specific model or brand of the television for which reviews are needed."
                    }
                },
                "required": ["model"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "CompareProducts",
            "description": "Compare different television models based on user-specified criteria such as price, size, or technology.",
            "parameters": {
                "type": "object",
                "properties": {
                    "criteria": {
                        "type": "object",
                        "properties": {
                            "price": {
                                "type": "string",
                                "description": "Price range for comparison."
                            },
                            "size": {
                                "type": "string",
                                "description": "Size of the televisions for comparison."
                            },
                            "technology": {
                                "type": "string",
                                "description": "Specific technology features to compare, like OLED, QLED, etc."
                            }
                        },
                        "description": "The criteria for comparing the television models."
                    },
                    "models": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "List of television models to compare."
                    }
                },
                "required": ["criteria", "models"]
            }
        }
    }
]

In [26]:
load_dotenv()  # import the environment variables
client = OpenAI(
  api_key=os.environ.get("OPENAI_API_KEY")
)

response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ],
  tools=tools,
  tool_choice="auto",
)

In [27]:
response_message = response.choices[0].message
tool_calls = response_message.tool_calls

In [29]:
print(response_message.content)
print(response_message)
print(tool_calls)

Choosing a TV involves considering several important factors to ensure that you select a model that best suits your needs. Here are some key considerations to keep in mind:

1. **Screen Size**: Determine the appropriate screen size based on the viewing distance and the size of the room where the TV will be placed. For example, for a 55-inch TV, a recommended viewing distance is about 7 feet.

2. **Resolution**: Choose between HD (1080p), 4K, or 8K resolution based on your content preferences and budget. Higher resolution provides sharper and more detailed images.

3. **Display Technology**: Decide between LED, OLED, QLED, and MicroLED. Each technology has its advantages, such as superior black levels, color accuracy, brightness, and contrast.

4. **Smart Features**: Look for smart TV capabilities with access to streaming services, voice control, and smart home integration.

5. **Refresh Rate**: For gamers and sports enthusiasts, a higher refresh rate (e.g., 120Hz or higher) can provide

for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    print(function_name)
    print(function_args)